# Mutrition Colaborative Filtering Recomendation System

Este agente recomendará recetas ordenadas, dependiendo de a que recetas le ha dado like el usuario basandose en los likes que han dado otros usuarios.

In [109]:
import pandas as pd
import difflib
import numpy as np
import pickle

In [110]:
#cargar dataset de recetas
recipes = pd.read_csv('recetas_procesadas.csv')
recipes.head(2)

,index,title,calories,protein,fat,sodium,vegan,huevos,frutas,marisco,lacteos,frutos secos,verduras,legumbres,carne,vegetariano,primer plato
0,0,"Lentil, Apple, and Turkey Wrap",426.0,30.0,7.0,559.0,0.0,0,1,0,0,0,1,0,1,0,1
1,1,Boudin Blanc Terrine with Red Onion Confit,403.0,18.0,23.0,1439.0,0.0,0,0,0,0,1,0,0,1,0,0


In [111]:
#cargar dataset de likes de usuarios a cada receta
user_recipe_likes = pd.read_csv('userlikes.csv')
user_recipe_likes.drop(columns=['Unnamed: 0'], inplace=True)
user_recipe_likes.head(3)

,userid,recetaid,like
0,0,1,0
1,0,3,0
2,0,7,1


# Creacion del Modelo

In [112]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [113]:
#Rating scale es el valor que puede dar un usuario a una receta, en nuetsro caso 1=like 0=noLike
#y las que no esten en la lista seran las recetas que no ha visto y que estaran entre las recomendadas
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(user_recipe_likes[['userid', 'recetaid', 'like']], reader)

In [114]:
#crear trainset
trainSet = data.build_full_trainset()

Crear el testset, este contiene los usuario con las recetas que no han votado.

In [115]:
anti_testset_user=trainSet.build_anti_testset()
# anti_testset_user = []
# targetUser = 0
# fillValue = trainSet.global_mean
# user_item_ratings = trainSet.ur[targetUser]
# user_items = [item for (item,_) in (user_item_ratings)]
# user_items
# ratings = trainSet.all_ratings()
# for iid in trainSet.all_items():
#   if(iid not in user_items):
#     anti_testset_user.append((trainSet.to_raw_uid(targetUser),trainSet.to_raw_iid(iid),fillValue))

# Normal Predictor

Probar con normal predictor

In [116]:
predictor = NormalPredictor()

In [117]:
predictor.fit(trainSet)
#cross_validate(predictor, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [118]:
predictions = predictor.test(anti_testset_user)

In [119]:
predictions[0]

Prediction(uid=0, iid=27, r_ui=0.49998282684261375, est=1, details={'was_impossible': False})

In [120]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
recipe_list = pred.head(5)['iid'].to_list()
recipes.loc[recipe_list]

,index,title,calories,protein,fat,sodium,vegan,huevos,frutas,marisco,lacteos,frutos secos,verduras,legumbres,carne,vegetariano,primer plato
27,31,Citrus Salad with Mint Sugar,191.0,3.0,1.0,4.0,1.0,0,1,0,0,0,0,0,0,1,0
9165,10474,"Watercress, Orange and Red Onion Salad",182.0,3.0,18.0,22.0,0.0,0,1,0,0,0,0,0,0,1,0
6972,8007,Garlic Soup,91.0,3.0,0.0,31.0,0.0,0,0,0,0,0,1,0,0,1,1
8710,9948,Rustic Green Pepper and Olive Pan Pizza,1588.0,75.0,107.0,2663.0,0.0,0,0,0,1,0,0,0,0,1,0
6959,7993,Ginger and Curry Leaf Rasam,340.0,9.0,26.0,321.0,1.0,0,0,0,0,0,0,1,0,1,1


# KNNeighbours
Probaremos con el algoritmo KNNeighbors (da mejores resultados, menor RMSE y menos MAE)

In [121]:
sim_options = {'name': 'cosine',
               'user_based': True  # False: similitud entre recetas | True: entre usuarios
               }
predictorKN = KNNBasic(sim_options=sim_options)
# Run 5-fold cross-validation and print results
cross_validate(predictorKN, data, measures=['RMSE', 'MAE'], cv=2, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    0.3624  0.3620  0.3622  0.0002  
MAE (testset)     0.2569  0.2568  0.2569  0.0000  
Fit time          10.24   15.46   12.85   2.61    
Test time         279.48  242.30  260.89  18.59   


{'test_rmse': array([0.36236506, 0.36200293]),
 'test_mae': array([0.25691408, 0.25683697]),
 'fit_time': (10.241899967193604, 15.463197231292725),
 'test_time': (279.4804120063782, 242.29549837112427)}

In [123]:
predictorKN.fit(trainSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [126]:
anti_testset_user = []
targetUser = 0
fillValue = trainSet.global_mean
user_item_ratings = trainSet.ur[targetUser]
user_items = [item for (item,_) in (user_item_ratings)]
user_items
ratings = trainSet.all_ratings()
for iid in trainSet.all_items():
  if(iid not in user_items):
    anti_testset_user.append((trainSet.to_raw_uid(targetUser),trainSet.to_raw_iid(iid),fillValue))

In [128]:
predictions = predictorKN.test(anti_testset_user)

In [135]:
predictions[0]

Prediction(uid=0, iid=27, r_ui=0.49998282684261375, est=0.8266766598538499, details={'actual_k': 40, 'was_impossible': False})

Listar las 10 recetas recomendadas en orden

In [139]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
recipe_list = pred.head(5)['iid'].to_list()
recipes.loc[recipe_list]

,index,title,calories,protein,fat,sodium,vegan,huevos,frutas,marisco,lacteos,frutos secos,verduras,legumbres,carne,vegetariano,primer plato
4859,5592,Tomato-and-Fennel-Stuffed Salmon with Basil Sa...,1377.0,97.0,99.0,1237.0,0.0,0,0,1,0,0,1,0,0,0,1
4549,5237,Roasted Broccoli with Garlic and Red Pepper,155.0,4.0,12.0,47.0,1.0,0,0,0,0,0,1,0,0,1,1
5432,6252,Macadamia and Ginger Stuffing,436.0,19.0,27.0,589.0,0.0,0,1,0,0,0,0,0,1,0,0
5156,5936,Roasted Potatoes and Cauliflower with Chives,239.0,5.0,11.0,29.0,1.0,0,0,0,0,0,1,0,0,1,1
607,710,Five-Spice Apple Quince Pie,650.0,5.0,27.0,161.0,0.0,0,1,0,1,0,0,0,0,1,0


Guardar el modelo ya fiteado en un fichero para no tener que ejecutar el comando cada vez

In [136]:
pickle.dump(predictorKN, open('modeloRecomenderKNN.sav', 'wb'))

Predecir que nota le daria un usuario a una receta, cuanto mas cerca del 1 mayor probabilidad de que le de like.

In [143]:
#usuario 5 a la receta 8000 
#deberia dar estimacion baja
prediction = predictorKN.predict(5, 8000, verbose=True)

#deberia dar alto
prediction = predictorKN.predict(600, 8000, verbose=True)

#deberia dar alto
prediction = predictorKN.predict(5, 50, verbose=True)

#deberia dar bajo
prediction = predictorKN.predict(600, 50, verbose=True)

user: 5          item: 8000       r_ui = None   est = 0.15   {'actual_k': 40, 'was_impossible': False}
user: 600        item: 8000       r_ui = None   est = 0.85   {'actual_k': 40, 'was_impossible': False}
user: 5          item: 50         r_ui = None   est = 0.95   {'actual_k': 40, 'was_impossible': False}
user: 600        item: 50         r_ui = None   est = 0.07   {'actual_k': 40, 'was_impossible': False}


In [156]:
estimaciones= []
for i in range(0, 10000):
   estimaciones.append(predictorKN.predict(1, i)[3])

estimaciones.sort(reverse=True)
estimaciones

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.9755954262524941,
 0.9755514824687173,
 0.9755470635795566,
 0.975488711708067,
 0.9754874581633298,
 0.9754547964172716,
 0.975443408867842,
 0.975426082845052,
 0.9754208306675446,
 0.9753967184698125,
 0.9753785288373574,
 0.9753784328537404,
 0.9753663645785734,
 0.9753456767185592,
 0.9753151085696606,
 0.9753073986736224,
 0.9752976616577573,
 0.9752813363199946,
 0.9752708630082434,
 0.9752703844055604,
 0.9752575629948962,
 0.9752437930001479,
 0.9752302830406884,
 0.9752211857782137,
 0.9752209927647515,
 0.9752188514049493,
 0.9751840347838772,
 0.9751787143335352,
 0.9751628546962305,
 0.9751559045974294,
 0.975137119718137,
 0.97513685827456,
 0.9751348765762834,
 0.9750977680125288,
 0.9750712144051926,
 0.9750652969039557,
 0.9750641672541994,
 0.9750395418381557,
 0.9750144937614338,
 0.9750114144982066,
 0.9749859744659253,
 0.9749804910109829,
 0.9749770124608619,
 0.9749698410811736,
 0.974